In [1]:
import polars as pl 
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import pyarrow.parquet as pq
import matplotlib.pyplot as plt

In [2]:
df = pl.read_parquet('/kaggle/input/tochka-bank-hack/train/train/ids_target.parquet.fastparquet')
df.head()


id,timestamp,client_id,seller_id,target
str,datetime[ns],str,str,i64
"""f90c6b1c-37fa-…",2020-06-22 00:00:00,"""7390515e-26ff-…","""241c7f98-838c-…",0
"""8bd5070f-541a-…",2023-05-02 20:00:00,"""8c2ad13a-90bd-…","""c7aa6d84-ffa9-…",0
"""0f57961f-677c-…",2022-11-08 16:00:00,"""2f5d044d-5d43-…","""3eeb1798-423b-…",0
"""16b48e53-7a87-…",2022-10-06 15:00:00,"""cce566a9-1f83-…","""3eeb1798-423b-…",0
"""df06c1d2-3601-…",2023-06-21 19:00:00,"""11d777ac-58cc-…","""668adc51-6344-…",0


In [5]:
client_gender=pl.read_parquet('/kaggle/input/tochka-bank-hack/train/train/prelead_features_client_gender.parquet.fastparquet')
client_hist=pl.read_parquet('/kaggle/input/tochka-bank-hack/train/train/prelead_features_client_history.parquet.fastparquet')
cbancs=pl.read_parquet('/kaggle/input/tochka-bank-hack/train/train/prelead_features_contractors_banks.parquet.fastparquet')
cdir=pl.read_parquet('/kaggle/input/tochka-bank-hack/train/train/prelead_features_contractors_directional.parquet.fastparquet')
egrul=pl.read_parquet('/kaggle/input/tochka-bank-hack/train/train/prelead_features_egrul.parquet.fastparquet')
zakup=pl.read_parquet('/kaggle/input/tochka-bank-hack/train/train/prelead_features_goszakupki.parquet.fastparquet')
holding=pl.read_parquet('/kaggle/input/tochka-bank-hack/train/train/prelead_features_holding.parquet.fastparquet')
revenue=pl.read_parquet('/kaggle/input/tochka-bank-hack/train/train/prelead_features_revenue.parquet.fastparquet')
time_f=pl.read_parquet('/kaggle/input/tochka-bank-hack/train/train/prelead_features_time_features.parquet.fastparquet')
user_info=pl.read_parquet('/kaggle/input/tochka-bank-hack/train/train/prelead_features_user_info.parquet.fastparquet')


In [5]:
files = [client_gender,client_hist,cbancs,cdir,egrul,zakup,holding,revenue,time_f,user_info]

In [6]:
for i, dfs in enumerate(files):
    suffix = f"_right_{i}"
    df = df.join(dfs, on = 'id', how= 'outer', suffix=suffix)

In [7]:
df.columns
df = df.drop(['id_right','id_right_0','id_right_1','id_right_2', 'id_right_3','id_right_4','id_right_5','id_right_6','id_right_7','id_right_8'
        ,'id_right_9'])

df.head()

id,timestamp,client_id,seller_id,target,client_gender,days_since_last_take,wrong_phone_last_time,has_new_phones,cnt_takes,cnt_not_sell,tochka_contractor_bank_top_1,tochka_contractors_n_incoming,tochka_contractors_count_sum_rub_incoming,tochka_contractors_avg_sum_rub_incoming,tochka_contractors_n_outgoing,tochka_contractors_count_sum_rub_outgoing,tochka_contractors_avg_sum_rub_outgoing,egrul_kind,egrul_business_type,egrul_region,egrul_reg_months_ago,goszakupki_winner_sum,goszakupki_winner_count,goszakupki_total_sum,goszakupki_total_count,cnt_holdings,days_since_ors_avg,is_youngest_in_holding,is_oldest_in_holding,is_biggest_in_holding,is_smallest_in_holding,holding_revenue_avg,holding_cnt_takes,holding_cnt_not_sell,holding_avg_days_since_take,revenue,time_tz_diff,role,division,position,grade,is_teamlead,teamid,hypothesisid,employed_days,seller_gender
str,datetime[ns],str,str,i64,str,i64,bool,bool,i32,i32,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,bool,bool,bool,bool,str,i64,i64,str,str,f64,str,str,str,str,bool,str,str,f64,str
"""7321ba8a-d0ba-…",2023-07-10 18:00:00,"""55ce0aab-102a-…","""4f606a99-6ca3-…",0,"""мужчина""",145,false,true,2,1,"""unknown""","""0""","""0""","""< 10000""","""0""","""0""","""< 10000""","""ЮЛ""","""unknown""","""region_71""","""0 - 6""","""0""","""0""","""0""","""0""","""0""","""unknown""",true,true,true,true,"""unknown""",0,0,"""unknown""","""unknown""",null,"""role_3""","""division_16""","""position_14""","""unknown""",false,"""a67e3b7b-5143-…","""1968a3b2-0e5e-…",3.0,"""мужчина"""
"""0d440614-cbbe-…",2020-09-02 11:00:00,"""39be8f21-27fd-…","""3eeb1798-423b-…",0,"""unknown""",9999,false,true,1,0,"""unknown""","""0""","""0""","""< 10000""","""0""","""0""","""< 10000""","""ИП""","""business_type_…","""region_29""","""0""","""0""","""0""","""0""","""0""","""0""","""unknown""",true,true,true,true,"""unknown""",0,0,"""unknown""","""unknown""",-2.0,"""role_9""","""division_8""","""position_33""","""unknown""",false,"""5701492f-edf4-…","""1968a3b2-0e5e-…",8.0,"""мужчина"""
"""9d43aa09-527c-…",2020-11-25 18:00:00,"""225c8aa5-cc82-…","""3c1d5392-9bb8-…",0,"""unknown""",9999,false,true,1,0,"""bank_281""","""0""","""0""","""< 10000""","""1 - 5""","""1 - 5""","""10000 - 100000…","""ЮЛ""","""business_type_…","""region_34""",""">36""","""1000000 - 1000…","""10 - 100""","""10000000-10000…",""">100""","""0""","""unknown""",true,true,true,true,"""unknown""",0,0,"""unknown""","""10000000 - 500…",1.0,"""role_5""","""division_2""","""position_22""","""unknown""",false,"""57ac9c83-b149-…","""9b57d9af-ac24-…",93.0,"""женщина"""
"""d045b012-0ce1-…",2021-02-15 12:00:00,"""1e4da912-ae35-…","""ea0578ad-3ffb-…",0,"""unknown""",9999,false,true,1,0,"""unknown""","""0""","""0""","""< 10000""","""0""","""0""","""< 10000""","""ИП""","""business_type_…","""region_64""","""0""","""0""","""0""","""0""","""0""","""0""","""unknown""",true,true,true,true,"""unknown""",0,0,"""unknown""","""unknown""",-1.0,"""role_9""","""division_8""","""position_33""","""grade_12""",false,"""a4206761-5a3d-…","""244722c4-755a-…",556.0,"""мужчина"""
"""69205b8d-c54f-…",2021-02-18 13:00:00,"""e9fc8261-f5bb-…","""ee311a07-f092-…",0,"""unknown""",9999,false,true,1,0,"""unknown""","""0""","""0""","""< 10000""","""0""","""0""","""< 10000""","""ЮЛ""","""business_type_…","""region_37""",""">36""","""0""","""0""","""0""","""0""","""1-5""","""unknown""",false,false,false,true,"""< 100000""",4,0,"""unknown""","""<10000000""",null,"""role_5""","""division_3""","""position_48""","""grade_4""",false,"""0c412d30-ed1b-…","""cc37ea06-3667-…",134.0,"""женщина"""


In [ ]:
df_test = pl.read_parquet('/kaggle/input/tochka-bank-hack/test/test/ids_target.parquet.fastparquet')
df_test.head(

In [8]:
client_gender_test=pl.read_parquet('/kaggle/input/tochka-bank-hack/test/test/prelead_features_client_gender.parquet.fastparquet')
client_hist_test=pl.read_parquet('/kaggle/input/tochka-bank-hack/test/test/prelead_features_client_history.parquet.fastparquet')
cbancs_test=pl.read_parquet('/kaggle/input/tochka-bank-hack/test/test/prelead_features_contractors_banks.parquet.fastparquet')
cdir_test=pl.read_parquet('/kaggle/input/tochka-bank-hack/test/test/prelead_features_contractors_directional.parquet.fastparquet')
egrul_test=pl.read_parquet('/kaggle/input/tochka-bank-hack/test/test/prelead_features_egrul.parquet.fastparquet')
zakup_test=pl.read_parquet('/kaggle/input/tochka-bank-hack/test/test/prelead_features_goszakupki.parquet.fastparquet')
holding_test=pl.read_parquet('/kaggle/input/tochka-bank-hack/test/test/prelead_features_holding.parquet.fastparquet')
revenue_test=pl.read_parquet('/kaggle/input/tochka-bank-hack/test/test/prelead_features_revenue.parquet.fastparquet')
time_f_test=pl.read_parquet('/kaggle/input/tochka-bank-hack/test/test/prelead_features_revenue.parquet.fastparquet')
user_info_test=pl.read_parquet('/kaggle/input/tochka-bank-hack/test/test/prelead_features_user_info.parquet.fastparquet')

In [9]:
test_files= [client_gender_test,client_hist_test,cbancs_test, cdir_test,egrul_test, zakup_test, holding_test,
             revenue_test,time_f_test, user_info_test ]

In [10]:
for i, dfs in enumerate(test_files):
    suffix = f"_right_{i}"
    df_test = df_test.join(dfs, on = 'id', how= 'outer', suffix=suffix)

In [11]:
df_test= df_test.drop(['id_right','id_right_0','id_right_1','id_right_2', 'id_right_3','id_right_4','id_right_5','id_right_6','id_right_7','id_right_8'
        ,'id_right_9'])
df_test.head()

id,timestamp,client_id,seller_id,client_gender,days_since_last_take,wrong_phone_last_time,has_new_phones,cnt_takes,cnt_not_sell,tochka_contractor_bank_top_1,tochka_contractors_n_incoming,tochka_contractors_count_sum_rub_incoming,tochka_contractors_avg_sum_rub_incoming,tochka_contractors_n_outgoing,tochka_contractors_count_sum_rub_outgoing,tochka_contractors_avg_sum_rub_outgoing,egrul_kind,egrul_business_type,egrul_region,egrul_reg_months_ago,goszakupki_winner_sum,goszakupki_winner_count,goszakupki_total_sum,goszakupki_total_count,cnt_holdings,days_since_ors_avg,is_youngest_in_holding,is_oldest_in_holding,is_biggest_in_holding,is_smallest_in_holding,holding_revenue_avg,holding_cnt_takes,holding_cnt_not_sell,holding_avg_days_since_take,revenue,revenue_right_8,role,division,position,grade,is_teamlead,teamid,hypothesisid,employed_days,seller_gender
str,datetime[ns],str,str,str,i64,bool,bool,i32,i32,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,bool,bool,bool,bool,str,i64,i64,str,str,str,str,str,str,str,bool,str,str,f64,str
"""ca28b95e-e53d-…",2024-01-29 14:00:00,"""7eafe7a5-28fe-…","""a944553d-fb9e-…","""unknown""",328,false,false,1,1,"""bank_265""","""1 - 5""","""1 - 5""","""< 10000""","""0""","""0""","""< 10000""","""ИП""","""business_type_…","""region_66""","""24 - 36""","""0""","""0""","""0""","""0""","""0""","""unknown""",true,true,true,true,"""unknown""",0,0,"""unknown""","""unknown""","""unknown""","""role_5""","""division_8""","""position_22""","""unknown""",false,"""2d863cc2-bff1-…","""94d95f5b-7380-…",154.0,"""женщина"""
"""842d0c3f-a390-…",2023-09-20 19:00:00,"""df3378ce-0347-…","""c105eabd-3742-…","""unknown""",225,false,true,2,0,"""unknown""","""0""","""0""","""< 10000""","""0""","""0""","""< 10000""","""ИП""","""business_type_…","""region_34""","""6 - 12""","""0""","""0""","""0""","""0""","""0""","""unknown""",true,true,true,true,"""unknown""",0,0,"""unknown""","""unknown""","""unknown""","""role_3""","""division_2""","""position_43""","""unknown""",false,"""046bf238-3844-…","""5aa4da89-7f72-…",-32.0,"""женщина"""
"""ef287b5b-f0d3-…",2023-10-04 22:00:00,"""ec0c604d-be52-…","""e91bd237-b421-…","""unknown""",9999,false,true,1,0,"""unknown""","""0""","""0""","""< 10000""","""0""","""0""","""< 10000""","""ИП""","""business_type_…","""region_76""","""0 - 6""","""0""","""0""","""0""","""0""","""0""","""unknown""",true,true,true,true,"""unknown""",0,0,"""unknown""","""unknown""","""unknown""","""role_9""","""division_1""","""position_51""","""unknown""",false,"""d536919b-7e7f-…","""0790e3e3-d4f3-…",292.0,"""женщина"""
"""d6fd08be-a74d-…",2024-02-26 13:00:00,"""47876573-f23a-…","""d08b52fa-e7d1-…","""unknown""",9999,false,true,1,0,"""unknown""","""0""","""0""","""< 10000""","""0""","""0""","""< 10000""","""unknown""","""business_type_…","""region_13""","""0 - 6""","""0""","""0""","""0""","""0""","""0""","""unknown""",true,true,true,true,"""unknown""",0,0,"""unknown""","""unknown""","""unknown""","""role_9""","""division_1""","""position_51""","""unknown""",false,"""d536919b-7e7f-…","""0790e3e3-d4f3-…",94.0,"""мужчина"""
"""449bb35b-079e-…",2023-12-25 13:00:00,"""3908730b-9263-…","""dc15b0ab-5569-…","""unknown""",9999,false,true,1,0,"""unknown""","""0""","""0""","""< 10000""","""0""","""0""","""< 10000""","""ИП""","""unknown""","""region_56""","""0 - 6""","""0""","""0""","""0""","""0""","""0""","""unknown""",true,true,true,true,"""unknown""",0,0,"""unknown""","""unknown""","""unknown""","""role_5""","""division_0""","""position_22""","""unknown""",false,"""b77f323f-c00f-…","""2c8b187f-6ebf-…",48.0,"""женщина"""


In [12]:
y = df['target']
df= df.drop('target')

In [13]:
df.columns == df_test.columns

False

In [14]:
missing_columns = set(df.columns) - set(df_test.columns)
print(missing_columns )

missing_columns = set(df_test.columns) - set(df.columns)
print(missing_columns )

{'time_tz_diff'}
{'revenue_right_8'}


In [15]:
df = df.drop('time_tz_diff')

df_test = df_test.drop('revenue_right_8')

In [16]:
df.columns == df_test.columns

True

In [17]:
df = df.with_columns(y)

In [ ]:
df['seller_gender'].value_counts()



In [ ]:
# df = df.drop(['days_since_ors_avg','holding_cnt_not_sell','holding_avg_days_since_take'])
# df_test = df_test.drop(['days_since_ors_avg','holding_cnt_not_sell','holding_avg_days_since_take'])

In [18]:
a = [df.select(pl.corr('tochka_contractors_count_sum_rub_incoming', 'target', method="spearman")),
df.select(pl.corr('tochka_contractors_avg_sum_rub_incoming', 'target', method="spearman")),
df.select(pl.corr('tochka_contractors_n_outgoing', 'target', method="spearman")),
df.select(pl.corr('tochka_contractors_count_sum_rub_outgoing', 'target', method="spearman")),
df.select(pl.corr('tochka_contractors_count_sum_rub_outgoing', 'target', method="spearman")),
df.select(pl.corr('tochka_contractors_avg_sum_rub_outgoing', 'target', method="spearman"))]

a 



[shape: (1, 1)
 ┌───────────────────────────────────┐
 │ tochka_contractors_count_sum_rub… │
 │ ---                               │
 │ f64                               │
 ╞═══════════════════════════════════╡
 │ 0.001925                          │
 └───────────────────────────────────┘,
 shape: (1, 1)
 ┌───────────────────────────────────┐
 │ tochka_contractors_avg_sum_rub_i… │
 │ ---                               │
 │ f64                               │
 ╞═══════════════════════════════════╡
 │ -0.003592                         │
 └───────────────────────────────────┘,
 shape: (1, 1)
 ┌───────────────────────────────┐
 │ tochka_contractors_n_outgoing │
 │ ---                           │
 │ f64                           │
 ╞═══════════════════════════════╡
 │ 0.006195                      │
 └───────────────────────────────┘,
 shape: (1, 1)
 ┌───────────────────────────────────┐
 │ tochka_contractors_count_sum_rub… │
 │ ---                               │
 │ f64                       

In [ ]:
# df = df.drop(['tochka_contractors_count_sum_rub_incoming','tochka_contractors_avg_sum_rub_incoming',
#              'tochka_contractors_n_outgoing','tochka_contractors_count_sum_rub_outgoing',
#              'tochka_contractors_count_sum_rub_outgoing','tochka_contractors_avg_sum_rub_outgoing'])
# df_test = df_test.drop(['tochka_contractors_count_sum_rub_incoming','tochka_contractors_avg_sum_rub_incoming',
#              'tochka_contractors_n_outgoing','tochka_contractors_count_sum_rub_outgoing',
#              'tochka_contractors_count_sum_rub_outgoing','tochka_contractors_avg_sum_rub_outgoing'])

In [ ]:
df


In [19]:
df = df.with_columns(df['wrong_phone_last_time'].cast(pl.Int64))
df = df.with_columns(df['has_new_phones'].cast(pl.Int64))
df = df.with_columns(df['is_youngest_in_holding'].cast(pl.Int64))
df = df.with_columns(df['is_oldest_in_holding'].cast(pl.Int64))
df = df.with_columns(df['is_biggest_in_holding'].cast(pl.Int64))
df = df.with_columns(df['is_smallest_in_holding'].cast(pl.Int64))
df = df.with_columns(df['is_teamlead'].cast(pl.Int64))

In [20]:
df_test = df_test.with_columns(df_test['wrong_phone_last_time'].cast(pl.Int64))
df_test = df_test.with_columns(df_test['has_new_phones'].cast(pl.Int64))
df_test = df_test.with_columns(df_test['is_youngest_in_holding'].cast(pl.Int64))
df_test = df_test.with_columns(df_test['is_oldest_in_holding'].cast(pl.Int64))
df_test = df_test.with_columns(df_test['is_biggest_in_holding'].cast(pl.Int64))
df_test = df_test.with_columns(df_test['is_smallest_in_holding'].cast(pl.Int64))
df_test = df_test.with_columns(df_test['is_teamlead'].cast(pl.Int64))

In [21]:
Y = df['target']
X = df.drop('target')

In [22]:
cat_features =[]
num_features =[]
for column in df.columns:
    if df[column].dtype in [pl.Int64, pl.Float64]:
        num_features.append(column)
    elif df[column].dtype == pl.Utf8:
        cat_features.append(column)

# TRAINING

In [23]:
X_train, X_test ,y_train, y_test = train_test_split(X,Y, test_size = 0.25, random_state = 42,  stratify=df['target'])

In [24]:
X_train = X_train.to_pandas()
y_train = y_train.to_pandas()
X_test, y_test = X_test.to_pandas(), y_test.to_pandas()

In [25]:
clf = CatBoostClassifier(iterations = 45, auto_class_weights='Balanced',cat_features = cat_features, learning_rate= 0.1)

In [26]:
clf.fit(X_train,y_train)


0:	learn: 0.6702221	total: 1.54s	remaining: 1m 7s
1:	learn: 0.6525430	total: 3.18s	remaining: 1m 8s
2:	learn: 0.6383107	total: 4.49s	remaining: 1m 2s
3:	learn: 0.6293351	total: 5.23s	remaining: 53.7s
4:	learn: 0.6205053	total: 6.29s	remaining: 50.4s
5:	learn: 0.6131814	total: 7.36s	remaining: 47.8s
6:	learn: 0.6036597	total: 8.59s	remaining: 46.6s
7:	learn: 0.5948663	total: 9.64s	remaining: 44.6s
8:	learn: 0.5881263	total: 10.7s	remaining: 42.9s
9:	learn: 0.5820536	total: 11.8s	remaining: 41.5s
10:	learn: 0.5774692	total: 13s	remaining: 40.3s
11:	learn: 0.5735386	total: 14.1s	remaining: 38.8s
12:	learn: 0.5698031	total: 15.2s	remaining: 37.3s
13:	learn: 0.5671290	total: 16.2s	remaining: 35.8s
14:	learn: 0.5641522	total: 17.2s	remaining: 34.5s
15:	learn: 0.5620358	total: 18.3s	remaining: 33.1s
16:	learn: 0.5599611	total: 19.3s	remaining: 31.9s
17:	learn: 0.5586633	total: 20.4s	remaining: 30.6s
18:	learn: 0.5573518	total: 21.5s	remaining: 29.4s
19:	learn: 0.5553973	total: 22.5s	remaining

In [27]:
y_pred = clf.predict(X_test)

f1_score(y_test, y_pred)

0.04355613585764743

In [ ]:
df_test = df_test.to_pandas()
answer = clf.predict(df_test)


In [ ]:
import pandas as pd

In [ ]:
final = pd.DataFrame({'id': df_test['id'] , 'target' : answer})

final
final.to_csv('final_answer.csv', index=False)